In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#read Bennet CDOM Master Transdrift with ALL ABSORPTIONS
Path_file_TD="/home/bjuhls/ownCloud/Paper_1/_work/CDOM-DOC_relationship/Data/original/ODV_Bennet_jens.csv"
data_TD=np.genfromtxt(Path_file_TD,delimiter=",", skip_header=1, usecols=range(0,641))
data_TD_col= zip(*data_TD)
header_read_td=np.genfromtxt(Path_file_TD,delimiter=",",usecols=range(0,641), dtype='str' )
header_td=header_read_td[0]
Str_Data = np.genfromtxt(Path_file_TD,delimiter=",",skip_header=1,usecols=range(0,641), dtype='str' )
Str_Data_col= zip(*Str_Data)

def cdomIndex(Ind):
    Num = Ind + 90
    return Num
print header_td[39]
print header_td[53]
def Slope(ref_wl, start_wl, end_wl, cdom_data):
    def fit_func_exp(x, a, b):
        return a * np.exp(-b * (x-ref_wl))
    x = range(start_wl, end_wl, 1)
    y = cdom_data[(start_wl-200):(end_wl-200)]
    params1_exp = curve_fit(fit_func_exp, x, y, method="lm", p0 = (cdom_data[ref_wl],0.02))
    return params1_exp[0][1]

def Slope_const(ref_wl, start_wl, end_wl, cdom_data):
    def fit_func_exp(x, a, b, c):
        return a * np.exp(-b * (x-ref_wl)) + c
    x = range(start_wl, end_wl, 1)
    y = cdom_data[(start_wl-200):(end_wl-200)]
    params1_exp = curve_fit(fit_func_exp, x, y, method="lm")
    #print params1_exp[0][2]
    return params1_exp[0][1]

def polyfit(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
     # Polynomial Coefficients
    results['polynomial'] = coeffs.tolist()
    # r-squared
    p = np.poly1d(coeffs)
    # fit values, and mean
    yhat = p(x)                         # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    results['determination'] = ssreg / sstot
    return results


Chl a (mg/m3)
Chl a-fluorescence [mg/m3]


In [2]:
abs_offset=[]
abs_raw=[]
Slopes_raw=[]
Slopes_off=[]
Slopes_const=[]
for line in data_TD[0:]:
    raw=line[cdomIndex(0):cdomIndex(550)]
    abs_raw.append(raw)
    off=line[cdomIndex(480)]
    abs_off= line[cdomIndex(0):cdomIndex(550)]-off
    abs_offset.append(abs_off)
    np_abs_offset=np.array(abs_offset)
    np_abs_raw=np.array(abs_raw)
    try:
        Slopes_raw.append(Slope(285,275,295,raw))
    except ValueError:
        Slopes_raw.append(np.nan)
    try:
        Slopes_off.append(Slope(285,275,295,abs_off))
    except ValueError:
        Slopes_off.append(np.nan)
    try:
        Slopes_const.append(Slope_const(285,275,295,raw))
    except (ValueError, RuntimeError) as e:
        Slopes_const.append(np.nan)
    np_Slopes_off=np.array(Slopes_off)
    np_Slopes_raw=np.array(Slopes_raw)
    np_Slopes_const=np.array(Slopes_const)
            
#except (IDontLikeYouException, YouAreBeingMeanException) as e:

In [11]:
### create new file and write data in it ### 

fp_datafile ='/home/bjuhls/ownCloud/Paper_1/Data/data_allStations.txt'
datafile = open(fp_datafile, 'w')

### wriing header
datafile.write("Event,Station,Date/Time,Latitude,Longitude,Depth [m],Temp [°C],Salinity [psu],d18O [per mil SMOW],DOC/NPOC [µmol/L],CDOM S275-295 [nm^-1],Chl a (mg/m3),Chl a-fluorescence [mg/m3],fa [%],fi [%],fr [%],")
for e in np.arange(200,750,1):
    datafile.write("aCDOM("+str(e) + ") [m^-1],")
datafile.write("\n")
datafile.close()

# opening file again and write each row
datafile = open(fp_datafile, 'a')

line_c=0 #row counter
for line in data_TD[0:]: # iterating through every row
    datafile.write(str(Str_Data[line_c][0]) + "," 
                    + str(Str_Data[line_c][1]) + "," 
                    + str(Str_Data[line_c][3]) + "," 
                    + str(data_TD[line_c][5]) + "," 
                    + str(data_TD[line_c][4]) + ","
                    + str(data_TD[line_c][7]) + "," 
                    + str(data_TD[line_c][9]) + "," 
                    + str(data_TD[line_c][11]) + "," 
                    + str(data_TD[line_c][15]) + ","
                    + str(data_TD[line_c][67]) + ","
                    + str(data_TD[line_c][39]) + ","
                    + str(data_TD[line_c][53]) + ","
                    + str(data_TD[line_c][41]) + ","
                    + str(data_TD[line_c][43]) + ","
                    + str(data_TD[line_c][45]) + ","
                    + str(Slopes_off[line_c]) + ","
                    + ",".join(map(str, abs_offset[line_c]))
                    ) # writing parameters which are stored in different indizies in each row
    datafile.write("\n") # writing absatz
        
    line_c+=1 # counting to next row
datafile.close()   # close data file

In [ ]:
# line_c=0
# for line in data_TD[0:]:
#     if line[150]>-1:
#         datafile.write(str(Str_Data[line_c][0]) + "," 
#                        + str(Str_Data[line_c][1]) + "," 
#                        + str(Str_Data[line_c][3]) + "," 
#                        + str(data_TD[line_c][5]) + "," 
#                        + str(data_TD[line_c][4]) + ","
#                        + str(data_TD[line_c][7]) + "," 
#                        + str(data_TD[line_c][9]) + "," 
#                        + str(data_TD[line_c][11]) + "," 
#                        + str(data_TD[line_c][15]) + ","
#                        + str(data_TD[line_c][67]) + ","
#                        + str(data_TD[line_c][39]) + ","
#                        + str(data_TD[line_c][53]) + ","
#                        + str(data_TD[line_c][41]) + ","
#                        + str(data_TD[line_c][43]) + ","
#                        + str(data_TD[line_c][45]) + ","
#                        + str(Slopes_off[line_c]) + ","
#                        + ",".join(map(str, abs_offset[line_c]))
#                       )
#         datafile.write("\n")
        
#     line_c+=1
# datafile.close()    
